In [1]:
import os
import shutil
from typing import Generator

class FileSystemOverlay:
    def __init__(self, base_dir: str, overlay_dir: str):

        self.base_dir = os.path.abspath(base_dir)
        self.overlay_dir = os.path.abspath(overlay_dir)

        os.makedirs(self.overlay_dir, exist_ok=True)

    def open(self, path: str, mode: str = 'r'):
       
        overlay_path = os.path.join(self.overlay_dir, path)
        base_path = os.path.join(self.base_dir, path)

        if 'r' in mode:
           
            if os.path.exists(overlay_path):
                return __builtins__.open(overlay_path, mode)
            elif os.path.exists(base_path):
                return __builtins__.open(base_path, mode)
            else:
                raise FileNotFoundError(f"File not found: {path}")
        elif 'w' in mode or 'a' in mode:
           
            os.makedirs(os.path.dirname(overlay_path), exist_ok=True)
            return __builtins__.open(overlay_path, mode)
        else:
            raise ValueError(f"Unsupported mode: {mode}")

    def list_dir(self, path: str) -> Generator[str, None, None]:
        
        overlay_path = os.path.join(self.overlay_dir, path)
        base_path = os.path.join(self.base_dir, path)

       
        entries = set()

        if os.path.exists(base_path):
            entries.update(os.listdir(base_path))

        if os.path.exists(overlay_path):
            entries.update(os.listdir(overlay_path))

        for entry in sorted(entries):
            yield entry

    def delete(self, path: str):

        overlay_path = os.path.join(self.overlay_dir, path)
        if os.path.exists(overlay_path):
            if os.path.isdir(overlay_path):
                shutil.rmtree(overlay_path)
            else:
                os.remove(overlay_path)
        else:
            raise FileNotFoundError(f"File or directory not found in overlay: {path}")

    def move(self, src: str, dest: str):
       
        src_path = os.path.join(self.overlay_dir, src)
        dest_path = os.path.join(self.overlay_dir, dest)

        if os.path.exists(src_path):
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            shutil.move(src_path, dest_path)
        else:
            raise FileNotFoundError(f"Source file or directory not found: {src}")

    def copy(self, src: str, dest: str):
        
        src_path = os.path.join(self.overlay_dir, src)
        dest_path = os.path.join(self.overlay_dir, dest)

        if os.path.exists(src_path):
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            if os.path.isdir(src_path):
                shutil.copytree(src_path, dest_path)
            else:
                shutil.copy2(src_path, dest_path)
        else:
            raise FileNotFoundError(f"Source file or directory not found: {src}")



In [2]:
# Example usage
if __name__ == "__main__":
    base = "base_dir"
    overlay = "overlay_dir"

    os.makedirs(base, exist_ok=True)
    example_base_file = os.path.join(base, "example.txt")

    with __builtins__.open(example_base_file, "w") as f:
        f.write("This is content from the base directory.")

    fs_overlay = FileSystemOverlay(base, overlay)

    # Example read operation
    try:
        with fs_overlay.open("example.txt", "r") as f:
            print("Read content:", f.read())
    except FileNotFoundError as e:
        print(e)

    # Example write operation
    with fs_overlay.open("new_file.txt", "w") as f:
        f.write("This is a new file in the overlay.")

    # Example list directory
    print("Directory contents:")
    for item in fs_overlay.list_dir("."):
        print(item)

    # Example move operation
    fs_overlay.move("new_file.txt", "moved_file.txt")

    # Example copy operation
    fs_overlay.copy("moved_file.txt", "copied_file.txt")


Read content: This is content from the base directory.
Directory contents:
copied_file.txt
example.txt
moved_file.txt
new_file.txt
